# Memory in CrewAI

#### Benefits of using Memory
* Adaptive Learning: Crews become more efficient over time, adapting to new information and refining their approach to tasks.
* Enhanced Personalization: Memory enables agents to remember user preferences and historical interactions, leading to personalized experiences.
* Improved Problem Solving: Access to a rich memory store aids agents in making more informed decisions, drawing on past learnings and contextual insights.

#### Short-Term Memory
Temporarily stores recent interactions and outcomes using RAG, enabling agents to recall and utilize information relevant to their current context during the current executions.
* Contextual Awareness: With short-term and contextual memory, agents gain the ability to maintain context over a conversation or task sequence, leading to more coherent and relevant responses.

#### Long-Term Memory	
Preserves valuable insights and learnings from past executions, allowing agents to build and refine their knowledge over time.
* Experience Accumulation: Long-term memory allows agents to accumulate experiences, learning from past actions to improve future decision-making and problem-solving.

#### Entity Memory	
Captures and organizes information about entities (people, places, concepts) encountered during tasks, facilitating deeper understanding and relationship mapping. Uses RAG for storing entity information.
* Entity Understanding: By maintaining entity memory, agents can recognize and remember key entities, enhancing their ability to process and interact with complex information.

#### Contextual Memory	
Maintains the context of interactions by combining ShortTermMemory, LongTermMemory, and EntityMemory, aiding in the coherence and relevance of agent responses over a sequence of tasks or a conversation.

#### External Memory	
Enables integration with external memory systems and providers (like Mem0), allowing for specialized memory storage and retrieval across different applications. Supports custom storage implementations for flexible memory management.

#### User Memory	
⚠️ DEPRECATED: This component is deprecated and will be removed in a future version. Please use External Memory instead.

## Implementing Memory in a Crew

#### By default, the memory system is disabled
By default, the memory system is disabled, and you can ensure it is active by setting `memory=True` in the crew configuration.

In [ ]:
from crewai import Crew, Agent, Task, Process

# Assemble your crew with memory capabilities
my_crew = Crew(
    agents=[...],
    tasks=[...],
    process=Process.sequential,
    memory=True,
    verbose=True
)

#### The memory will use OpenAI embeddings by default
The memory will use OpenAI embeddings by default, but you can change it by setting `embedder` to a different model.

#### Where are short-term and long-term memories stored?
* The ‘embedder’ only applies to Short-Term Memory which uses Chroma for RAG.
* The Long-Term Memory uses SQLite3 to store task results.
* **Currently, there is no way to override these storage implementations**.

#### Where in your computer are saved this memory files?
The data storage files are saved into a platform-specific location found using the `appdirs` package, and the name of the project can be overridden using the CREWAI_STORAGE_DIR environment variable. This means:

1. **Platform-specific location using `appdirs`**:
   - CrewAI saves its data (e.g., config files, cache, etc.) in a standard directory that depends on your operating system (Windows, macOS, Linux).
   - This directory is determined using the [`appdirs`](https://pypi.org/project/appdirs/) Python package, which helps applications find appropriate locations for storing data, like:
     - macOS: `~/Library/Application Support/<AppName>`
     - Linux: `~/.local/share/<AppName>`
     - Windows: `C:\Users\<User>\AppData\Local\<AppName>`

2. **Override with `CREWAI_STORAGE_DIR`**:
   - If you don’t want CrewAI to use the default directory chosen by `appdirs`, you can set the environment variable `CREWAI_STORAGE_DIR` to specify a custom folder path.
   - This is useful if you want to control where CrewAI keeps its files — for example, to use a shared network drive, keep things isolated for testing, or maintain a clean directory structure.

Here’s how you can set the `CREWAI_STORAGE_DIR` environment variable on different operating systems:

**Windows (Command Prompt)**

```cmd
set CREWAI_STORAGE_DIR=C:\path\to\your\custom\folder
```

> This only lasts for the session. To make it permanent, you can set it via:
- **Control Panel > System > Advanced system settings > Environment Variables**
- Under "User variables", click **New**:
  - Name: `CREWAI_STORAGE_DIR`
  - Value: `C:\path\to\your\custom\folder`

**macOS or Linux (bash/zsh shell)**

To set it for the current session:
```bash
export CREWAI_STORAGE_DIR=/path/to/your/custom/folder
```

To make it permanent, add the line above to your shell config file:
- `~/.bashrc` or `~/.bash_profile` for **bash**
- `~/.zshrc` for **zsh**

Then run:
```bash
source ~/.bashrc   # or ~/.zshrc
```

Once this variable is set, CrewAI will store its data in the directory you specified.

## How to use custom Memory Instances like FAISS as the vectorDB

In [ ]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from typing import List, Optional

# Assemble your crew with memory capabilities
my_crew: Crew = Crew(
    agents = [...],
    tasks = [...],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="/my_crew1/long_term_memory_storage.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config={
                    "provider": "openai",
                    "config": {
                        "model": 'text-embedding-3-small'
                    }
                },
                type="short_term",
                path="/my_crew1/"
            )
        ),
    ),
    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config={
                "provider": "openai",
                "config": {
                    "model": 'text-embedding-3-small'
                }
            },
            type="short_term",
            path="/my_crew1/"
        )
    ),
    verbose=True,
)

## Let's explain the previous code
The previous code sets up a `Crew` with multiple types of memory: **long-term**, **short-term**, and **entity memory**, each configured with a custom storage backend. Let’s break it down line-by-line and clarify how **FAISS** or **alternatives** come into play.

#### Key Components

1. **Memory Types Used**
CrewAI supports memory for agents:
- `LongTermMemory`: For persistent memory across sessions.
- `ShortTermMemory`: For handling current context during interactions (like working memory).
- `EntityMemory`: For remembering facts about specific entities (e.g., a person or company).

2. **Storage Backends**
Each memory type can plug into a **storage engine**. These engines determine where and how embeddings and knowledge are stored and queried.

The two shown here are:
- `LTMSQLiteStorage`: SQLite-based storage for **long-term memory**.
- `RAGStorage`: RAG = Retrieval-Augmented Generation. This is often where **FAISS** or vector DBs come in.

#### Imports

```python
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
```
This imports the memory-related components and two storage backends: one for long-term (SQLite) and one for short-term/entity (RAG).

#### Setting up the Crew
```python
my_crew: Crew = Crew(
    agents = [...],
    tasks = [...],
    process = Process.sequential,
    memory = True,
```
This initializes a `Crew` object, enables memory, and sets it to work sequentially through tasks.

#### Long-Term Memory
```python
long_term_memory = LongTermMemory(
    storage=LTMSQLiteStorage(
        db_path="/my_crew1/long_term_memory_storage.db"
    )
)
```
- Uses `LTMSQLiteStorage` to persist data.
- **No FAISS here**—just plain SQLite.

#### Short-Term and Entity Memory
```python
short_term_memory = ShortTermMemory(
    storage = RAGStorage(
        embedder_config={
            "provider": "openai",
            "config": {
                "model": 'text-embedding-3-small'
            }
        },
        type="short_term",
        path="/my_crew1/"
    )
)
```
- Uses `RAGStorage` for embedding-based retrieval.
- The `embedder_config` defines how to convert text to vectors (OpenAI embeddings here).
- **This is where FAISS is used internally**: `RAGStorage` stores and retrieves vectors, and it uses FAISS under the hood.

Same logic applies to `EntityMemory`.

#### Where Is FAISS Used?

**FAISS is not explicitly imported or configured in this code.** However:
- `RAGStorage`'s implementation uses FAISS as the vector store backend, then **FAISS is used internally**.

## Security Considerations
* Use environment variables for storage paths (e.g., CREWAI_STORAGE_DIR).
* Never hardcode sensitive information like database credentials.
* Consider access permissions for storage directories.
* Use relative paths when possible to maintain portability.

## CrewAI allows integrating Mem0 for Enhanced User Memory

[Mem0](https://mem0.ai/) is an open-source, self-improving memory layer designed for Large Language Model (LLM) applications, aiming to enable personalized and context-aware AI experiences. It addresses the stateless nature of traditional LLMs by providing mechanisms to remember and learn from user interactions over time.

#### What Is Mem0?

Mem0 functions as an intelligent memory system for AI applications, allowing them to:

- **Retain User Preferences**: Mem0 stores information about user interactions, enabling AI systems to recall past conversations and preferences.

- **Adapt to Individual Needs**: By learning from ongoing interactions, Mem0 helps AI applications tailor responses to individual users. 

- **Continuously Improve**: The system evolves with each interaction, refining its understanding and enhancing the user experience over time.

These capabilities are particularly beneficial for applications like customer support chatbots, personal AI assistants, and autonomous systems.

#### How Mem0 Works

Mem0 employs a hybrid datastore architecture that combines:

- **Graph Stores**: To capture relationships between different pieces of information.

- **Vector Stores**: For efficient similarity searches based on embeddings.

- **Key-Value Stores**: To quickly retrieve specific data points. 

This architecture allows Mem0 to manage various types of memory, including:

- **Long-Term Memory**: Persistent information retained across sessions.

- **Short-Term Memory**: Contextual information relevant to the current session.

- **Semantic Memory**: Understanding of concepts and facts.

- **Episodic Memory**: Recollection of specific events or interactions.

The system is designed to be easily integrated into existing AI applications, with support for various platforms and frameworks .

#### Performance Highlights

According to recent research, Mem0 demonstrates significant improvements over traditional memory systems:

- **Accuracy**: Achieves 26% higher accuracy compared to OpenAI's memory on the LOCOMO benchmark.

- **Latency**: Delivers responses 91% faster than full-context approaches.

- **Token Efficiency**: Reduces token usage by 90%, lowering operational costs.

These enhancements make Mem0 a compelling choice for developers seeking to build more efficient and personalized AI applications .

#### Integration and Use Cases

Mem0 can be integrated into AI applications through its SDKs or APIs. Developers can choose between a hosted platform or a self-hosted open-source package. Use cases include:

- **Customer Support**: Chatbots that remember past interactions to provide more accurate assistance.

- **Personal AI Companions**: Assistants that adapt to user preferences over time.

- **Healthcare Applications**: Systems that track patient history for personalized care.

- **Productivity Tools**: Applications that learn from user behavior to optimize workflows.

## Miscelanea: Using Mem0 in a Crew, etc.

In [ ]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = "m0-xx"

# Step 1: Create a Crew with User Memory

crew = Crew(
    agents=[...],
    tasks=[...],
    verbose=True,
    process=Process.sequential,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
        "user_memory" : {} #Set user_memory explicitly to a dictionary, we are working on this issue.
    },
)

## Miscelanea: Mem0 possibilities, Embedder Options, etc.